Uncompress and split laz/las in multiple las files with specified target size. Assumes points to be uniform distributed over space to determine how many rows/columns the original file will be split into.

In [1]:
import laspy
import numpy as np
import numba as nb
import os

In [2]:
@nb.njit()
def find_rowcol(value, intervals):
    """
    helper function for lasz_to_smaller_las
    """
    for rowcol, intval in enumerate(intervals):
        if value <= intval:
            return rowcol
    print("NO ROW OR COLUMN FOUND??")
    return None

@nb.njit(parallel=True)
def get_row_col(xpoints, ypoints, row_x_intervals, col_y_intervals):
    """
    helper function for lasz_to_smaller_las
    """
    n_points = len(xpoints)
    xpoints_rows = np.zeros(n_points, dtype=np.int64)
    ypoints_cols = np.zeros(n_points, dtype=np.int64)
    for i in nb.prange(n_points):
        xpoints_rows[i] = find_rowcol(xpoints[i], row_x_intervals)
        ypoints_cols[i] = find_rowcol(ypoints[i], col_y_intervals)
    
    return xpoints_rows, ypoints_cols

@nb.njit(parallel=True)
def points_mask_from_row_col(xpoints_rows, ypoints_cols, row, col):
    """
    helper function for lasz_to_smaller_las
    """
    n_points = len(xpoints_rows)
    mask = np.zeros(n_points, dtype=np.bool_)
    for i in nb.prange(n_points):
        if xpoints_rows[i] == row and ypoints_cols[i] == col:
            mask[i] = True
    return mask

def lasz_to_smaller_las(fname_in, fname_out, max_out_MB, chunksize):
    """
    - max target size in megabytes
    - assumes points to be uniformly distributed over x and y
    - creates new las files with _row_col appended to fname_out
    """
    # determine the amount of rows and columns to split the original file in
    in_MB = os.path.getsize(fname_in)/1000000
    n_col = 1
    n_row = 1
    counter = 0
    while in_MB/(n_col*n_row) > max_out_MB:
        counter += 1
        if counter % 2 == 1:
            n_row += 1
        else:
            n_col += 1
    print("file in MB / (columns * rows) = files out MB")
    print(f"{in_MB} / ({n_col}*{n_row}) = {in_MB/(n_col*n_row)}")
    
    with laspy.open(fname_in) as src:
        # get x y min max values of src
        x_min = src.header.x_min
        x_max = src.header.x_max
        y_min = src.header.y_min
        y_max = src.header.y_max
    
        # init files to write to
        for row in range(n_row):
            for col in range(n_col):
                name = fname_out.split(".")[-2]+f"_{row}_{col}.las"
                dst_file = laspy.create(point_format=src.header.point_format, file_version=src.header.version)
                dst_file.write(name)

    # construct intervals
    x_range = x_max - x_min
    y_range = y_max - y_min
    x_step = x_range / n_row
    y_step = y_range / n_col
    row_x_intervals = [] # upper values for the rows
    col_y_intervals = [] # upper values for the cols
    for i in range(n_row):
        row_x_intervals.append((i+1)*x_step + x_min)
    for i in range(n_col):
        col_y_intervals.append((i+1)*y_step + y_min)
    row_x_intervals = np.array(row_x_intervals, dtype=np.float64)
    col_y_intervals = np.array(col_y_intervals, dtype=np.float64)

    # open src file and iterate over points per chunk
    with laspy.open(fname_in) as src:
        src_point_count = src.header.point_count
        counter = 0
        for points in src.chunk_iterator(chunksize):
            
            # get points x and y values
            xpoints = np.array(points.x, dtype=np.float64)
            ypoints = np.array(points.y, dtype=np.float64)
        
            # determine which row and column the points are in
            xpoints_rows, ypoints_cols = get_row_col(xpoints, ypoints, row_x_intervals, col_y_intervals)
        
            # write points to corresponding row/column file
            for row in range(n_row):
                for col in range(n_col):
                    points_mask = points_mask_from_row_col(xpoints_rows, ypoints_cols, row, col)
                    points_towrite = points[points_mask]
                    if len(points_towrite) != 0:
                        name = fname_out.split(".")[-2]+f"_{row}_{col}.las"
                        with laspy.open(name, mode='a') as dst:
                            dst.append_points(points_towrite)
                    
            counter += 1
            if (counter*chunksize)/src_point_count < 1:
                print(f"{round((counter*chunksize)/src_point_count, 2)} done")
            else:
                print("ENTIRELY DONE")

    return

# Run

In [3]:
fname_in = "C_64EZ2_pred.las"
fname_out = "C_64EZ2_pred_smol.las"
max_out_MB = 250
chunksize = 10_000_000

lasz_to_smaller_las(fname_in, fname_out, max_out_MB, chunksize)

file in MB / (columns * rows) = files out MB
1411.435569 / (2*3) = 235.2392615
0.2 done
0.4 done
0.6 done
0.79 done
0.99 done
ENTIRELY DONE
